# Hyper-parameter Search Project

Note: This is a work in progress and documentation is not complete.

## Introduction to project

### Background

Most machine learning applications require hyper-parameter tuning.  The typical approach is to input a range of values for each hyper-parameter, the search creates a grid of all of the combinations of the differnet hyper-parameters, then the model is solved for each of the hyper-parameter combinations in the grid and some metric is use to pick the "best" model.  Regardless, hyper-parameter tuning can require significant computing resources because the expectation is to try a large number models with the expectation of picking the best one.  If a model takes 1 minute to run, but there are 60 hyper-parameter combinations, then the comput time would be 1 hr.  For continuous hyper-parameters, e.g., regularization parameter, it is possible to build an adaptive search, but that is beyond the scope of this project.

There are a number of different strategies for picking the "best" model.  In machine learning the most common approach is cross-validation (CV), where the data is split into x folds, and there are x repititions of using x-1 folds to train a model and using the left out fold to evaluate the fit.  The left out fold rotates in each repitition so that a prediction is made for each data point, but this approach requires fitting several models for each hyper-parameter combination in the grid, and increases computation time.

In inverse modeling, the cross validation approach can't be employed, so other approaches are used.  Akaike information criterion (AIC), Bayesian information criterion (BIC), generalized cross validation (GCV), and un-biased predictive risk estimator (UPRE) are examples approaches the can be used for different types of problems.  GCV and UPRE can be very expensive to compute because they require explicitly computing a pseudo-inverse and influence matrix so approximation methods, e.g., http://brendt.wohlberg.net/publications/pdf/lin-2010-upre.pdf, may be required for large date sets.


### Project Overview

The goal of this project is to demonstrate the importance of hyper-paramter tuning.  A second goal it to show how to actually implement a hyper-parameter tuning strategy (many people just count on `sklearn.model_selection.gridsearchcv()` to do most of the work).

This project uses a classic denoising problem to demonstrate the importance of hyper-parameter tuning.  This type of problem uses a regularized inversion to estimate  the true data values from a noisy time series.  The solution of this type of problem is almost wholly dependent on the regularization parameter value used, and to lesser extent the order of the finite difference operator employed in the regularization.  Other problems show a strong dependence on the hyper-parameters used, but this problem goes from a useless fit to a nearly perfect fit back to a useless fit over the range of hyper-parameters used.

This type of problem doesn't lend itself to using CV to select the best model.  A typical linear regression can use 4-folds to parameterize the model and then evaluate the fit of the predictions on the left out fold.  Basic denoising problems don't create a continous function in the same way as linear regression, so predicting data points that are not measured isn't really a possiblity without interpolating existing data points; this type of problem simply smoothes the data points in existing timeseries without parameterizing some prediction function.  


This project creates a noisy data time series and attempt to de-noise it using regularized smoothing.  Because `gridsearchcv()` can't be used to do the hyper-parameter search for this type of problem, the grid search is explicitly implemented here.  The script converts the input hyper-paramter values into a grid, loops through that grid, and evaluates each model using the GCV metric; GCV is meant to approximate leave-one-out (LOO) cross-validation.  All of the models fits are stored and an interactive plot is provided so the user can explore just how dependent the fit is on the regularization parameter value used.


### Project Structure

The project is structured into five sections: generating forward data, code to run the de-noising algorithm, creating the hyper-parameter grid and fitting the de-noising model to the data, code to create the visualizations, and the interactive implementation.  The sections need to be run in order.  The forward data is generated and model fitting is completely run and stored prior to visualization.  Because The model must be completely fit to create the scoring plots, the interactive graphs at the end simply reference the stored models and do not call functions to compute model fits in real time.  

Scroll to the bottom of this page use this algorithm, or slowly work your way down to see how it works.

In [1]:
#preamble
from ipywidgets import interact
import ipywidgets as widgets

import numpy as np
from numpy.random import default_rng

from scipy import sparse as sp

import itertools
import math
import pprint

from tqdm import tqdm
tqdm._instances.clear()

#sets the number of jobs to parallel process
#-1 corresponds to the maximum number of cores available
from joblib import Parallel, delayed
n_jobs_use=-1

from functools import cached_property

import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import pandas as pd



## Forward Model Data

This section creates and plots some forward model data.  The data is based on a sine wave, with noised added to it, and includes the option to vary the amplitude of the wave over the data set.  `cycles`, `num_data_points`, `noise_level`, `y_coeff`,and `y_scaling` can be adjusted by the user to change the nature of the forward data.  Overall, a base sine wave with `num_data_points` is generated, this sine wave is skewed using `y_scaling`, the overall magnitude is adjusted with `y_coeff`, and normally distributed noise with standard deviation `noise_level` is added.  The plot at this section overlays the true data set on top of the noisy data set.

In [ ]:
#forward data

#BEGIN USER INPUT SECTION

#number of cycles in the sine wave
cycles=2

#number of data points to evaluate the sine wave function
num_data_points=2_000

#specifies the standard deviation of the normaly distrbuted noise to add to data
noise_level=1

#multiplier to apply to the base data, effectively plays against noise_level
y_coeff=1

#linear multiplier for the data set so that the magnitude of the sine wave is not constant
#across the data set
y_scaling=np.linspace(1,5,num_data_points)


#creates the x input vector to go cycles number of sine wave cycles
#with num_data_points number of data points
x=np.linspace(0,2*math.pi*cycles,num_data_points)

#END USER INPUT SECTION

#evaluates the sine function on the x vector and adjusts the magnitude with the scaling vector
y_base= y_scaling * np.sin(x)

#creates random normal noise with a fixed seed 
#and adds noise to the y-data set after adjusting the noise-less data with y_coeff
rng = default_rng(0)
y_noise=rng.normal(loc=0,scale=noise_level,size=y_base.shape)
y=y_coeff * y_base + y_noise


#creates a plot to show the noiseless and noisy data
#I have found that lines work better than dots for this type of scatter plot
#because the noisy data effectively creates a solid background upon which to view the noiseless data
#the color scheme here is the same as the color scheme later
fig={}
fig['base_data_plot']=make_subplots(rows=1, cols=1)
#fig_base_data.update_layout(height=4 * 96 * 1, width=9.5 * 96 * 1)
fig['base_data_plot'].add_scatter(x=x,y=y,name='y',mode='lines')
fig['base_data_plot'].add_scatter(x=x, y=y_base, name = 'y true', mode='lines', line=dict(width=3, color='white'))
#fig['base_data_plot'].update_layout(plot_bgcolor='white')

#the default plot color (fig['base_data_plot']['layout']['plot_bgcolor']) is white, so I am not sure why it shows as a blue-grey
fig['base_data_plot'].update_layout(plot_bgcolor='lightgrey')
#print(fig['base_data_plot']['layout']['plot_bgcolor'])
fig['base_data_plot'].update_layout(legend=dict(bgcolor=fig['base_data_plot']['layout']['plot_bgcolor']))
#fig['base_data_plot'].update_layout(legend=dict(bgcolor=None))
#fig['base_data_plot'].show()


## Minimization Function and Solver

This section contains the code to solve the de-noising problem and to compute the "score" for each of the candidate models in the hyper-parameter tuning set.  

### Minimization Problem

The minimization function to solve is
$$
\min_\hat{d} \dfrac{1}{2} \left\|\hat{d}-d\right\|^2_2 + \dfrac{\lambda}{2} \left\|D_k \hat{d}\right\|^2_2,
$$
where $d$ is the noisy data, $\hat{d}$ is the reconstruction of the true data, and $D_k$ is a finite difference operator of order $k$, e.g.,
$$
D_1=
\begin{bmatrix}
-1 &  1 & 0 &       & \\
 0 & -1 & 1 & 0     & \\
   &    &   &       & \ddots
\end{bmatrix}
$$
and
$$
D_2=
\begin{bmatrix}
1 & -2 &  1 & 0 &       & \\
0 &  1 & -2 & 1 & 0     & \\
  &    &    &   &       & \ddots
\end{bmatrix}.
$$
Dimensions on $D_k$ are $\left[\left(n-k\right) x\ n\right]$.


To simplify solving this problem we can re-write it as 
$$
\min_\hat{d} \dfrac{1}{2}
\left\|
\begin{bmatrix}
I_n\\
\sqrt{\lambda}D_k
\end{bmatrix} \hat{d}
-
\begin{bmatrix}
d\\
0_{n-k}
\end{bmatrix}
\right\|^2_2
$$
where $I_n$ is the identity matrix corresponding to the length of $d$ and $0_{n-k}$ is a column vector of zeros with length $n-k$, the same number of rows as in $D_k$.

### Model Selection

Because this model can not predict new data in the same manner as a non-linear regression, the CV method for model selection can not be used here.  Instead we use the GCV method, and pick the model that has the minimum value for $S_\lambda$, where
$$
S_\lambda
=
\dfrac{
\dfrac{1}{n}
\left\|R_\lambda\right\|^2_2
}
{
\left[
\dfrac{1}{n}
\text{trace}
\left(
I_n-T_\lambda
\right)
\right]^2
}
$$
where are the model residuals at a given $\lambda$ value, $R_\lambda=\hat{d}-d$, and $T_\lambda$ is the corresponding influence matrix
$$
T_\lambda
=
A \left(A^T A + \lambda R^T R\right)^{-1} A^T,
$$
which for this problems simplifies to 
$$
T_\lambda
=
I_n \left(I_n + \lambda {D_k}^T D_k\right)^{-1} {I_n}^T.
$$

### Section Structure

This section contains code to compute a trailing finite difference operator based on an arbitary stencil.  This section begins by computing the stencil based on the order of the finite difference operator, computing the finite difference coefficients, and creates an appropriately dimensioned diagonal band matrix with these coefficients.  A series of hierarchical classes are created to solve the general regularized least squares problem and then layer in L2 total variation regularization and the denoising problem.

In [ ]:
def finite_diff_coeff(s,d):
#https://en.wikipedia.org/wiki/Finite_difference_coefficient
#finite difference coefficient vector
#s is the stencil points, e.g., [-1,0,1], as a row vector
#d is the derivative order
#retuns a column vector
    
    n=len(s)
    LHS=np.power(s,np.arange(n).reshape(n,1))
    RHS=math.factorial(d)*np.concatenate([np.zeros(d),np.array(1,ndmin=1),np.zeros(n-d-1)])
    coeff=np.linalg.solve(LHS,RHS).reshape(1,n)
    
    return coeff
    
def trailing_small_finite_diff_matrix(k,order=1,n=1):
    #https://en.wikipedia.org/wiki/Finite_difference_coefficient
    #k = the length of the dataset
    #order is the order fo the finite difference derivative
    #n is the number of stencil points, and relates to the derivative order and desired accuracy
    
    #print(k)
    
    #ensures num stencil points is n>=d+1
    #for this application typically use n=d+1
    n=max(n,order+1)
    #print(n)
    
    #computes trailing stencil, end is exclusive so use 1 instead of 0
    #this is the simplest solution that scales well
    #since we only want the sum of the derivatives, and we don't care about the derivative at a single point, 
    #the actual indexing of the derivative is irrelevant
    s=np.arange(-n+1,1)
    #print(s)
    
    #compute trailing finite difference coefficeints
    D_coeff=finite_diff_coeff(s,order)
    #print(D_coeff)
    
    #puts everything into band matrix
    #this is almost always a sparse matrix and benefits from sparse implementation
    D=sp.spdiags((np.dot(np.ones((k,1)),D_coeff)).T,range(n), k-(n-1), k)
    #D=sp.spdiags((np.dot(np.ones((k,1)),D_coeff)).T,range(n), k-(n-1), k).todense()
    
    return D

class L2_resid_L2_reg():
    
    #this class solves L2 residual - L2 regularization problem for linear inversion/regression
    #takes a linear operator, regularization operator, and regularization parameter and fits
    #the model and returns the parameter values
    #currently it also returns the Generalize Cross Validation (GCV score), but that can be expanded
    
    def __init__(self,A,R,lam=0, store_score_matrix=False):
        
        #initializes the model with the
        #A=linear operator
        #R=regularization operator (I am not sure that it supports no regularization at this point)
        #lam=regularization parameter
        #store_score_matrix = boolean to say whether or not to store matrices associated with the GCV score
            #these matrices can take up a lot of memory (large full matrices) so sometimes long term storage isn't feasible
        
        self.A=A
        self.R=R
        self.lam=lam
        self.store_score_matrix=store_score_matrix
        #debug()
        
        #return self doesn't play well with the super class
       # return self
    
    def fit(self,y):
        #fits the regularized linear model to data set y
        
        #A = linear blurring operator
        #y = true data
        #R = linear regularization operator
        #lam=regularization parameter value
        
        self.y=y

        #debug()
        #if one or both of the matrices is full, the operator_stack is full
        #otherwise operator_stack is sparse, assumption is if one of the operators
        #is full then it is for a reason
        #could add logic to check the expected sparsity of keep sparse
        #sp.vstack((full,sparse))=sparse
        #this would be much easier in MATLAB
        if isinstance(self.A,np.ndarray):
            #full A, convert R to full
            self.operator_stack=np.vstack((self.A,np.sqrt(self.lam)*self.R.A))
            
        elif isinstance(self.A,np.ndarray):
            #full R, convert A to full
            self.operator_stack=np.vstack((self.A.A,np.sqrt(self.lam)*self.R))
            
        elif isinstance(self.A,np.ndarray) and isinstance(self.F,np.ndarray):
            #both full, no conversion
            self.operator_stack=np.vstack((self.A,np.sqrt(self.lam)*self.R))
        else:
            #both sparse, no conversion
            self.operator_stack=sp.vstack((self.A,np.sqrt(self.lam)*self.R))
        
        #assumption is b is full
        self.b=np.concatenate((self.y,np.zeros((self.R.shape[0]))))
        #print(lam)
        
        #stores the shape of the matrix as [p x q]
        self.p,self.q=self.A.shape
    
        #print(operator_stack.shape)
        #print(y.shape)
        #print(np.zeros((R.shape[0])).shape)
        #print(np.concatenate((y,np.zeros((R.shape[0])))).shape)
    
        
        #solves the least squares problem using lstsq or lsqr
        #I think these functions use a QR or related decomposition and
        #benefit from not explicitly forming a Hessian matrix which squares the condition number
        
        #ignoring the 1/2, which would go in lsqr as np.sqrt(1/2)
        #doing this changes the objective function value, but not the location of the minimum
        if isinstance(self.operator_stack, np.ndarray):
            #full operator_stack
            self.full_model=np.linalg.lstsq(self.operator_stack,self. b)
        else:  
            #sparse operator_stack
            self.full_model=sp.linalg.lsqr(self.operator_stack,self.b)#,x0=y)
        #full_model=sp.linalg.lsqr(np.sqrt(1/2)*operator_stack,np.sqrt(1/2)*b);
        #full_model=np.linalg.lstsq(operator_stack.A,b)  #slow
        
        #stores the model fit and condition number in explicit attributes
        self.m=self.full_model[0]
        self.acond=self.full_model[6]
        
        #evaluates the predicted data and the regularization vector
        self.yhat = self.A @ self.m
        self.Reg_m = self.R @ self.m
        
        #computes residuals and the norm of the residuals
        self.resid = self.yhat-y
        self.norm_resid = np.linalg.norm(self.resid)
        
        return self
    #print(y.shape)
    #print(model['yhat'].shape)
    
    def pseudo_inv_fun(self):
        
        #function to compute the pseudo_inv for the regularized linear problem
        #only use this in evaluating fit score because of potential for 
        #numerical instability
        
        #put this in it's own function so I can call it for the cached property function
        #as well as the memory shortcut application without having to duplicate all of the if-thens
        
        #using sparse solver because the LHS is sparse
        #even though both inputs are sparse the output of this will be full
        #converting the RHS to full here is actually faster than if I kept it as sparse
        #i.e., sp.linalg.spsolve(operator_stack.T @ operator_stack, A.T)
        #and overall this is faster than doing all of the calculations as sparse and converting to full
        #i.e., sp.linalg.spsolve(operator_stack.T @ operator_stack, A.T).todense()
        #this is what I had observed in matlab
        #model['pseudo_inv']= sp.linalg.spsolve(operator_stack.T @ operator_stack, A.A.T)
        
        
        if isinstance(self.operator_stack,np.ndarray):
            #operator_stack is full, A is sparse
            #A needs to be full for np.linalg.solve()
            return np.linalg.solve(self.operator_stack.T @ self.operator_stack, self.A.A.T)
        
        elif isinstance(self.A,np.ndarray):
            #operator_stack is sparse, A is already full
            #which if good for sp.linalg.spsolve, but A.A doesn't exist if A is already full
            return sp.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.T)
                
        elif isinstance(self.operator_stack,np.ndarray) and isinstance(self.A,np.ndarray):
            #both operator_stack and A are full
            return np.linalg.solve(self.operator_stack.T @ self.operator_stack, self.A.T)
        else:   
            #both operator_stack and A are sparse
            #if A is sparse then this solves faster if convert A to full first
            #also sp.linalg.spsolve(sparse,full)=full, which is better for later computations
            return sp.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.A.T)
    
    #@cached_property computes and stores class attributes on demand
    #the first time the attribute is called it is computed and then stored as an attribute
    #https://stackoverflow.com/questions/1598174/pythonic-way-to-only-do-work-first-time-a-variable-is-called
    #the function name becomes the attribute name
    
    @cached_property
    def pseudo_inv(self):
        #computes and stores the pseudo inverse matrix (AtA + lam*RtR)*At
        #evaluates and stores the pseudo inverse from pseudo_inv_fun in self.pseudo_inv
        #this can take up a lot of memory so sometimes it should be avoided
        
        #return sp.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.A.T)
        return self.pseudo_inv_fun()
    

    @cached_property
    def influence_matrix(self):
        #computes and stores the influence matrix A*(AtA + lam*RtR)*At
        #this can take up a lot of memory so sometimes it should be avoided
        
        #here sparse @ full = full and is faster than full @ full
        #I am not sure if this is what I saw in Matlab, and it may not scale well
        #for a less sparse A, but it works here
        #model['influence_matrix'] = A.A @ model['pseudo_inv']
        #model['influence_matrix'] = A @ model['pseudo_inv']
        
        #if self.store_score_matrix:
        return self.A @ self.pseudo_inv
    
    @cached_property
    def GCV_val(self):
        #debug()
        #computes and stores the Generalized Cross Validation (GCV) score value
        
        if self.store_score_matrix:
            #uses stored values for influence_matrix and by proxy for the pseudo inverse
            #doint this can take up a lot of memory
            local_influence_matrix=self.influence_matrix
        else:
            #calculates pseudo inverse and influence_matrix but does not store them
            #lose calculated information, but requires less memory for large datasets
            
            local_influence_matrix=self.A @ self.pseudo_inv_fun()
            
            # #pseudo_inv=sp.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.A.T)
            # #influence_matrix=self.A @ pseudo_inv
            # if isinstance(self.operator_stack,np.ndarray):
            #     #operator_stack is full, A is sparse
            #     #if A is sparse it is faster to convert A to full
            #     local_influence_matrix=self.A @ np.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.A.T):
            
            # elif isinstance(self.A,np.ndarray):
            #     #operator_stack is sparse, A is already full
            #     local_influence_matrix=self.A @ sp.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.T):
                    
            # elif isinstance(self.operator_stack,np.ndarray) and isinstance(self.A,np.ndarray):
            #     #both operator_stack and A are full
            #     local_influence_matrix=self.A @ np.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.T):
            # else:   
            #     #both operator_stack and A are sparse
            #     #if A is sparse it is faster to convert A to full
            #     local_influence_matrix=self.A @ sp.linalg.spsolve(self.operator_stack.T @ self.operator_stack, self.A.A.T)
            
        #debug()
        return 1/self.p * self.norm_resid ** 2 \
                /(1-np.trace(local_influence_matrix)/self.p) ** 2
                #/(1/model['p'] * np.trace(np.eye(len(y))-model['influence_matrix'])) ** 2 is
                #slower because you have to form I, do the full subtraction, and then take the trace
                #trace(I_p) is just p
                
    
    def GCV_val_calc(self):
        #this is a function call version of GCV_val for use with joblib.parallel
        return self.GCV_val
    #debug()
    
class L2_L2_TV_debulurring(L2_resid_L2_reg):
    #class the for the case of L2_resid_L2_reg where you are using L2 D_n @ m style regularization
    def __init__(self,A,order=1,lam=0, store_score_matrix=False):
        
        #initializes a model a L2_resid_L2_reg with finite difference regularization
        #finite difference order is set in order
        #this type of problem can be used for L2-TV based de-noising and deblurring
        
        #gets shape of linear operator for residual portion of problem
        #uses this shape to create appropriately sized regularization operator
        p,q=A.shape
        
        #create appropriately sized regularization operator
        #this is sparse diagonal band matrix
        R=trailing_small_finite_diff_matrix(k=p,order=order)
        
        
        #initiates this class by calling the parent class-L2_resid_L2_reg
        #sends the linear operator, regularization operator, and lambda value to the parent class
        super().__init__(A,R,lam,store_score_matrix)
        
       # return self
        

class L2_L2_TV_denoising(L2_L2_TV_debulurring):
    #class the for the case of L2_resid_L2_reg where you are using
    #L2 I @ m resiguals and
    #L2 D_n @ m style regularization
    
    def __init__(self,p,order=1,lam=0, store_score_matrix=False):
        
        #initializes model by layering de-blurring operator on top of 
        #L2-TV regularization
        #in this class we specify a de-blurring operator that will be sent to
        #the de-blurring/de-noising class whose parent is the base L2-L2 solver class
        
        #creates sparse identity matrix to match size of input data
        #this is the linear operator
        A=sp.eye(p)

        #initiates this class by calling the parent class-L2_L2_TV_debulurring
        #sends the linear operator, finite difference operator order, and lambda value to the parent class
        super().__init__(A,order,lam,store_score_matrix)
        
        #return self

In [ ]:
#generates and runs the hyper parameter search

min_log_lam=-15
max_log_lam=15
step_log_lam=0.5
#str(step_log_lam)[::-1].find('.') reverses the order and then counts characters to the decimal, finds the number of decimal places
log_lam=np.arange(min_log_lam,max_log_lam+10*np.spacing(max_log_lam),step_log_lam).round(str(step_log_lam)[::-1].find('.'))

min_finite_diff_order=1
max_finite_diff_order=2
#finite_diff_operator={'D%d'%(i): trailing_small_finite_diff_matrix(k=len(y),order=i) 
 #                     for i in range(min_finite_diff_order,max_finite_diff_order+1)}
finite_diff_operator_order={'D%d'%(i): i for i in range(min_finite_diff_order,max_finite_diff_order+1)}

#combines lambda and the finite difference operator into a parameter grid
#this variable is just for display
#parameter_grid_dict_display={'log_lam':list(log_lam),
 #                            'R': finite_diff_operator.keys()}
parameter_grid_dict_display={'log_lam':list(log_lam),
                             'R': finite_diff_operator_order.keys()}
    

#this variable is for actual use
#parameter_grid_dict={'lam':list(10**log_lam),
 #                    'R': finite_diff_operator.values()}
parameter_grid_dict={'lam':list(10**log_lam),
                     'order': finite_diff_operator_order.values()}


def parameter_grid_to_list_of_param_dict(param_grid):
    keys, values = zip(*param_grid.items())
    return [dict(zip(keys, v)) for v in itertools.product(*values)]

parameter_grid_list_display=parameter_grid_to_list_of_param_dict(parameter_grid_dict_display)
parameter_grid_list=parameter_grid_to_list_of_param_dict(parameter_grid_dict)

#prints the dictionary of list
pprint.pprint(parameter_grid_dict_display,sort_dicts=False, compact=True)
print()
#prints the list of dictionaries
pprint.pprint(parameter_grid_list_display,sort_dicts=False, compact=True)

#model_list=[solve_denoising(sp.eye(len(y)),y,**reg_params) for reg_params in tqdm(parameter_grid_list)]
model_list=[L2_L2_TV_denoising(len(y),**reg_params,store_score_matrix=False).fit(y) for reg_params in tqdm(parameter_grid_list)]
#I have been tinkering with implementing parallel delayed for this type of user defined class 
#but I haven't had much luck so far.  I have gotten a lot of #PicklingError: Could not pickle the task to send it to the workers.
#On Binder I am not sure if parallel processing would make much difference, but I would like to figure this out
#https://stackoverflow.com/questions/50528331/parallel-class-function-calls-using-python-joblib
#https://stackoverflow.com/questions/50528331/parallel-class-function-calls-using-python-joblib
#model_list_base=[L2_L2_TV_denoising(len(y),**reg_params,store_score_matrix=False) for reg_params in tqdm(parameter_grid_list)]
#model_list=Parallel(n_jobs=n_jobs_use)(delayed(model.fit)(y) for model in tqdm(model_list_base))


#work around to deal with problem of not being able to load class values,
#model_list_dict=[model.__dict__ for model in model_list]

In [ ]:
#pivots results into two level dictionary 
#with 1) difference operator type, 2) log reg prameter
#this is not a general solution 

model_dict_plotting={finite_diff_operator_name:
                     {setup_dict['log_lam']:model for setup_dict,model in zip(parameter_grid_list_display,model_list) 
                      if setup_dict['R']==finite_diff_operator_name}
                     for finite_diff_operator_name in finite_diff_operator_order.keys()}

In [ ]:
#functions for plotting

#plt.plot([model['GCV_val'] for model in model_dict_plotting['D2'].values()])

#score_plot_arr=np.array([[model['GCV_val'] for model in model_dict_plotting[diff_operator_name].values()] 
 #                        for diff_operator_name in model_dict_plotting.keys()]).T
score_plot_arr=np.array([[model.GCV_val for model in tqdm(model_dict_plotting[diff_operator_name].values())] 
                         for diff_operator_name in model_dict_plotting.keys()]).T

score_plot_arr.shape

#START HERE

#print({key: score_plot_arr[:,i] for i, key in enumerate(finite_diff_operator)})

base10_plot_df=pd.DataFrame({'x': log_lam, **{key: score_plot_arr[:,i] for i, key in enumerate(finite_diff_operator_order)}})
log10_plot_df=pd.DataFrame({'x': log_lam, **{key: np.log10(score_plot_arr[:,i]) for i, key in enumerate(finite_diff_operator_order)}})

def plot_x_y(diff_operator_name, log_lam_val):
    
    plot_x_y_df=pd.DataFrame({'x':x,
                              'y':y,
                              'y hat': model_dict_plotting[diff_operator_name][round(log_lam_val,str(step_log_lam)[::-1].find('.'))].yhat,
                              'y true': y_base})
                              #'y hat': model_dict_plotting[diff_operator_name][log_lam_val]['yhat']})
    
    fig['denoised_data_plot']=make_subplots(rows=1, cols=1)
    #fig.update_layout(height=8 * 96 * 1, width=9.5 * 96 * 1, )
    
    #fig = px.add_scatter(plot_x_y_df, x='x', y='y',
     #               labels={'x':'x', 'value':'y', "variable": "data type"}).update_traces(mode='lines')
    fig['denoised_data_plot'].add_scatter(x=plot_x_y_df['x'], y=plot_x_y_df['y'],name = 'y',mode='lines')#,line=dict(color='blue'))
    
    fig['denoised_data_plot'].add_scatter(x=plot_x_y_df['x'], y=plot_x_y_df['y hat'], name = 'y hat', mode='lines', line=dict(width=4))#, color='red'))
    
    fig['denoised_data_plot'].update_layout(plot_bgcolor='lightgrey')
    #print(fig['base_data_plot']['layout']['plot_bgcolor'])
    fig['denoised_data_plot'].update_layout(legend=dict(bgcolor=fig['denoised_data_plot']['layout']['plot_bgcolor']))
    
    #marker_spacing=3
    #fig.add_scatter(x=plot_x_y_df['x'][marker_spacing-1::marker_spacing], y=plot_x_y_df['y true'][marker_spacing-1::marker_spacing], 
     #               name = 'y true', mode='markers', marker=dict(color='white',
      #                                                           size=5,
       #                                                          line=dict(color='black',
        #                                                                   width=0.5)))
                    
    fig['denoised_data_plot'].add_scatter(x=plot_x_y_df['x'], y=plot_x_y_df['y true'], name = 'y true', mode='lines',line=dict(width=2, color='white',dash = 'dash'))
    

    fig['denoised_data_plot'].show()
    return

#init_guess_ind=np.argmin(np.array([model['GCV_val'] for model in model_list]))
init_guess_ind=np.argmin(np.array([model.GCV_val for model in model_list]))


In [ ]:
#does the plotting and the interaction

#plot_df
plot_df=base10_plot_df
#plot_df=log10_plot_df
fig['score_plot']=px.scatter(plot_df,
                             x='x', 
                             y=plot_df.columns[1:],
                             labels={'x':'log10 lambda', 'value':'GCV score', "variable": "Diff Op"}).update_traces(mode='lines')
#fig1.update_layout(height=4 * 96 * 1, width=9.5 * 96 * 1)
fig['score_plot'].update_layout(plot_bgcolor='lightgrey')
print(fig['base_data_plot']['layout']['plot_bgcolor'])
fig['score_plot'].update_layout(legend=dict(bgcolor=fig['score_plot']['layout']['plot_bgcolor']))
fig['score_plot'].show()
#model_dict_plotting.keys()

log_lam_widget=widgets.FloatSlider(min=min_log_lam, max=max_log_lam, step=step_log_lam, 
                                   value=parameter_grid_list_display[init_guess_ind]['log_lam'],
                                   description='log10 lambda:',)
diff_operator_widget=widgets.Dropdown(options=model_dict_plotting.keys(),
                                      value=parameter_grid_list_display[init_guess_ind]['R'],
                                      description='Finite diff operator:')

interact(plot_x_y,
        diff_operator_name=diff_operator_widget,
        log_lam_val=log_lam_widget)

print('The last plot can be very touchy in Binder; it works consistently locally in JupyterLab.')
print('If the plot is not updating when you change the input values re-run the last cell.')

